## Detailed Code Explanation

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541544/fine-tuning-google-gemma-model-for-text-classification-in-python

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Installing and Importing Required Libraries

In [8]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
!pip3 install huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 55.9 MB/s eta 0:00:00


In [28]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

## Testing the Gemma Model for Casual LM Tasks

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
model_id = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [13]:
text = "Jack of all"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Jack of all trades, master of none.

That’s


## Importing and Preprocessing the Dataset

In [14]:
dataset = pd.read_csv(r"/content/IMDB Dataset.csv")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset = dataset.head(5000)
print(dataset['sentiment'].value_counts())
dataset.head()

negative    2528
positive    2472
Name: sentiment, dtype: int64


,review,sentiment
0,This movie was release when I was 15 and I cou...,positive
1,"Before I review this film, I must make a confe...",positive
2,i really liked the film.at ending i was in tea...,positive
3,"I have been a ""huge"" rush fan ever since ""movi...",positive
4,"......this film is pretty awful, the only thin...",negative


In [15]:
dataset = Dataset.from_pandas(dataset)
final_dataset = dataset.train_test_split(test_size=0.2)

In [16]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 1000
    })
})

In [17]:
def formatting_func(example):
    text = f"Review: {example['review'][0]}\nSentiment: {example['sentiment'][0]}"
    return [text]

In [18]:
formatting_func(final_dataset['train'])

['Review: i was very impressed with this production on likely all levels; from production to plot and character development.<br /><br />this definitely fall under the "realism" genre, since there is nothing going on here that makes use of creative imagination, twists, or manipulating audience in how a viewer shall feel, think and asses.<br /><br />the actors are great, especially the "little" ones. the chemistry between the brother and the sister is mind blowing, maybe not even as common as should be in real life. it is a movie excellent to literally watch while visualizing it. one knows when a movie is good when one just does not want it to end, but eventually everything does...\nSentiment: positive']

## Fine Tuning Google Gemma Model

In [19]:
os.environ["WANDB_DISABLED"] = "false"

In [20]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [21]:
trainer = SFTTrainer(
    model=model,
    train_dataset=final_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [22]:
trainer.train()

Step,Training Loss
1,3.147200
2,3.147200
3,3.120200
4,3.055800
5,2.984000
6,2.907300
7,2.822100
8,2.736900
9,2.646300
10,2.550400


TrainOutput(global_step=100, training_loss=0.6013533681258559, metrics={'train_runtime': 201.9394, 'train_samples_per_second': 1.981, 'train_steps_per_second': 0.495, 'total_flos': 1284640204800000.0, 'train_loss': 0.6013533681258559, 'epoch': 100.0})

In [23]:
print(final_dataset['test'][2])

{'review': 'If you know anything about the Manhattan Project, you will find "Fat Man and Little Boy" at least an interesting depiction of the events surrounding that story. The film is in all ways a very realistic portrayal of these events, and in many ways it is almost too real (such as some scenes involving radiation poisoning). Paul Newman, as usual, is brilliant in his role and always manages to come off like a real person on the screen. The supporting cast, such as John Cusack, Laura Dern, Bonnie Bedelia, and Natasha Richardson, is fairly good as well. This film is not, however, one of the best examples of turning a true story into a movie. Great films are able to take a true story and use just enough artistic license to keep its audience engaged for the entire movie. This one, however, tends to drag a bit throughout, and some scenes (such as John Cusack and Natasha Richardson\'s love story) could have been eliminated entirely without causing the film to lose much. Nevertheless, t

In [24]:
text = f"Review: {final_dataset['test'][2]['review']}\nSentiment: "
print(text)

Review: If you know anything about the Manhattan Project, you will find "Fat Man and Little Boy" at least an interesting depiction of the events surrounding that story. The film is in all ways a very realistic portrayal of these events, and in many ways it is almost too real (such as some scenes involving radiation poisoning). Paul Newman, as usual, is brilliant in his role and always manages to come off like a real person on the screen. The supporting cast, such as John Cusack, Laura Dern, Bonnie Bedelia, and Natasha Richardson, is fairly good as well. This film is not, however, one of the best examples of turning a true story into a movie. Great films are able to take a true story and use just enough artistic license to keep its audience engaged for the entire movie. This one, however, tends to drag a bit throughout, and some scenes (such as John Cusack and Natasha Richardson's love story) could have been eliminated entirely without causing the film to lose much. Nevertheless, there 

In [25]:
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=1)
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
prediction.split(" ")[-1]

'positive'

## Evaluating Fine-tuned Model Performance on Test Set

In [26]:
def predict_sentiment(review):
  text = f"Review: {review}\nSentiment: "
  inputs = tokenizer(text, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=1)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  sentiment = prediction.split(" ")[-1]
  return sentiment


In [27]:
targets = []
predictions = []

for i in range(len(final_dataset['test'])):

  review = final_dataset['test'][i]['review']
  target_sentiment = final_dataset['test'][i]['sentiment']
  predicted_sentiment = predict_sentiment(review)

  if predicted_sentiment in ["positive", "negative"]:
    targets.append(target_sentiment)
    predictions.append(predicted_sentiment)
    print(f"Record {i+1} - Actual:{target_sentiment}, Predicted: {predicted_sentiment}")


Record 1 - Actual:negative, Predicted: negative
Record 2 - Actual:negative, Predicted: negative
Record 3 - Actual:positive, Predicted: positive
Record 4 - Actual:negative, Predicted: negative
Record 5 - Actual:negative, Predicted: negative
Record 6 - Actual:negative, Predicted: negative
Record 7 - Actual:negative, Predicted: negative
Record 8 - Actual:positive, Predicted: negative
Record 9 - Actual:positive, Predicted: negative
Record 10 - Actual:positive, Predicted: negative
Record 11 - Actual:positive, Predicted: positive
Record 12 - Actual:negative, Predicted: positive
Record 13 - Actual:positive, Predicted: positive
Record 14 - Actual:positive, Predicted: negative
Record 15 - Actual:positive, Predicted: positive
Record 16 - Actual:positive, Predicted: positive
Record 17 - Actual:positive, Predicted: positive
Record 18 - Actual:positive, Predicted: negative
Record 19 - Actual:negative, Predicted: negative
Record 20 - Actual:positive, Predicted: negative
Record 21 - Actual:positive, 

In [29]:
# Calculate accuracy
accuracy = accuracy_score(targets, predictions)
print(f'Accuracy: {accuracy:.2f}')

# Generate classification report
report = classification_report(targets, predictions)
print('Classification Report:\n', report)

Accuracy: 0.88
Classification Report:
               precision    recall  f1-score   support

    negative       0.82      0.98      0.89       472
    positive       0.98      0.78      0.87       478

    accuracy                           0.88       950
   macro avg       0.90      0.88      0.88       950
weighted avg       0.90      0.88      0.88       950

